# Installation

In [0]:
%pip install tabula-py pandas tabulate azure-core azure-search-documents pypdf openai

In [0]:
%pip install openai

In [0]:
%pip install PyPDF2

# Creating Search Index

In [0]:
import os
import glob
import json
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchIndex,
    ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField
)

import tabula
from tabulate import tabulate
from pypdf import PdfReader as pypdfReader
from PyPDF2 import PdfReader as PyPDF2Reader
from PyPDF2 import PdfWriter as PyPDF2Writer



In [0]:
#data = {'@search.action':search_action, 'id': str(i), 'content_table':markdown, 'page_number':page_number, 'document_link_web': pdf_info[3], 'document_link_storage': pdf_info[0],#'dataType': dataType, 'title' : title , 'content_html':df_html , 'content_text' : text }

In [0]:
schema_obj = r'''{
    "name": "index-tables-from-pdf-bahamas-v4",  
    "fields": [
        {"name": "id", "type": "Edm.String", "key": true, "filterable": true},
        {"name": "content", "type": "Edm.String", "searchable": true, "filterable": false, "sortable": true, "facetable": false},
        {"name": "page_number", "type": "Edm.String", "searchable": true, "filterable": false, "sortable": true, "facetable": false},
        {"name": "document_link", "type": "Edm.String", "searchable": true, "filterable": false, "sortable": true, "facetable": false},
        {"name": "document_link_storage", "type": "Edm.String", "searchable": true, "filterable": false, "sortable": true, "facetable": false},
        {"name": "dataType", "type": "Edm.String", "searchable": true, "filterable": false, "sortable": true, "facetable": false},
        {"name": "title", "type": "Edm.String", "searchable": true, "filterable": false, "sortable": true, "facetable": false},
        {"name": "content_html", "type": "Edm.String", "searchable": true, "filterable": false, "sortable": true, "facetable": false},
        {"name": "content_text", "type": "Edm.String", "searchable": false, "filterable": false, "sortable": true, "facetable": false},
        {"name": "document_filename", "type": "Edm.String", "searchable": true, "filterable": false, "sortable": true, "facetable": false}
  ]
}'''
       
schema_json = json.loads(schema_obj)
      
print(schema_json)

In [0]:
service_name = "wedocumentsearchdemocase"
admin_key = ""
index_name = "index-tables-from-pdf-bahamas-v4"

In [0]:
# Create an SDK client
endpoint = "https://{}.search.windows.net/".format(service_name)
admin_client = SearchIndexClient(endpoint=endpoint,
                       index_name=index_name,
                       credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                       index_name=index_name,
                       credential=AzureKeyCredential(admin_key))

for i in range(92,93):
    print(i)
    data = {'@search.action':"delete", 'id': str(i)}
    print(data)
    data_upload = json.dumps(data)
    indexDocument = json.loads(data_upload) 
    try:
        result = search_client.delete_documents(documents=indexDocument)
        print("Delete of document succeeded: {}".format(result[0].succeeded))
    except Exception as ex:
        print (ex.message)

In [0]:
name = index_name

fields = schema_json["fields"]

cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)

scoring_profiles = []
suggester = []

In [0]:
index = SearchIndex(
    name=name,
    fields=fields,
    scoring_profiles=scoring_profiles,
    suggesters = suggester,
    cors_options=cors_options
    )

try:
    result = admin_client.create_index(index)
    print ('Index', result.name, 'created')
except Exception as ex:
    print (ex)

In [0]:
# Constants
storage_account_name = "storage2023jti"
storage_account_key = ""
container = "containerjti"

# Set Azure Data Lake Storage configuration
spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "SharedKey")
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

In [0]:
pdf_path = "/dbfs/FileStore/bahamas_hurricane_kit_2023.pdf"

In [0]:
path_silver = "abfss://testcontainer@storage2023jti.dfs.core.windows.net/metadata_silver_delta"

df_silver = spark.read.format("delta").load(path_silver)

In [0]:
df_silver.show()

In [0]:
df = tabula.read_pdf(pdf_path, pages = 4, guess = False)

In [0]:
print(df)

In [0]:
#dbutils.fs.mkdirs("dbfs:/FileStore/pdfs_temp_for_text/")

In [0]:
#dbutils.fs.rm("dbfs:/FileStore/pdfs_single_page/", True)

In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/pdfs_single_page/")

In [0]:
dbfs_pdf_path ="dbfs:/FileStore/pdfs_temp_for_text/"

In [0]:
#add function to split pdf up in pages

In [0]:
def save_pdf_page_to_storage(file_path, page_number):
    try:
        #reader = pypdfReader(file_path)
        print(page_number)
        pdf = PyPDF2Reader(file_path)
        pdfWriter = PyPDF2Writer()
        # The page numbering in pdfWriter starts at 0
        pdfWriter.add_page(pdf.pages[page_number-1])
        filename = pdf_info[0].rsplit('/', 1)[-1]
        #remove the .pdf or .ashx and add .pdf, so browser can open in app
        filename = filename.rsplit('.', 1)[0]
        with open('/dbfs/FileStore/pdfs_single_page/' + 'page_{0}'.format(str(page_number)) + '_{0}'.format(filename) + '.pdf', 'wb') as f:
            pdfWriter.write(f)
            f.close()
        return('page_{0}'.format(str(page_number)) + '_{0}'.format(filename))
    except Exception as ex:
                print (ex.message)

In [0]:
def convert_pdf_page_to_text(file_path, pages):
    try:
        reader = pypdfReader(file_path)
        page = reader.pages[pages]
        text = "\n" + page.extract_text() + "\n"
        #print(text)
        if len(text) > 0:

            return text
        else:
            return None
    except Exception as ex:
                print (ex.message)

# Test
#df_text = convert_pdf_page_to_text(pdf_path, 4)
#if df_text:
    #print(df_text)
#else:
    #print("No text were converted in the PDF.")

In [0]:
def convert_pdf_table_to_html(file_path, pages):
    # Read table from PDF
    try:
        df = tabula.read_pdf(file_path, pages=pages, silent=True)
        if len(df) > 0:
            df = df[0]  # Extract the first table
        
            df_html = df.to_html()
            return df_html
        else:
            return None
    except Exception as ex:
                print (ex.message)

# Test
df_html = convert_pdf_table_to_html(pdf_path, 5)
if df_html:
    print(df_html)
else:
    print("No tables were converted in the PDF.")

In [0]:
def convert_pdf_table_to_md(file_path, pages):
    try:
        # Read table from PDF
        df = tabula.read_pdf(file_path, pages=pages)
        if len(df) > 0:
            df = df[0]  # Extract the first table

            # Convert DataFrame to Markdown
            markdown = tabulate(df, tablefmt="pipe", headers="keys")
            return markdown
        else:
            return None
    except Exception as ex:
                print (ex.message)

# Test
markdown = convert_pdf_table_to_md(pdf_path, 5)
if markdown:
    print(markdown)
else:
    print("No tables were found in the PDF.")

In [0]:
def copy_pdf_from_filestore_to_blob(filename):
    # Constants
    storage_account_name = "storage2023jti"
    storage_account_key = ""
    container = "containerjti"

    # Set Azure Data Lake Storage configuration
    spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "SharedKey")
    spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

    # Define the local path to the PDF file
    dbfs_pdf_path = "dbfs:/FileStore/pdfs_single_page/"
#/dbfs/FileStore/pdfs_single_page/page_1_bahamas_hurricane_kit_2023.pdf
    # Define the path for the output PDF in Azure Blob Storage
    blob_pdf_path = "abfss://testcontainer@storage2023jti.dfs.core.windows.net/pdfs_single_page"

    # Use dbutils.fs to copy the PDF file to Azure Blob Storage
    dbutils.fs.cp(dbfs_pdf_path + filename, blob_pdf_path)

    # Print the paths for verification
    print("PDF file copied to Azure Blob Storage at:", blob_pdf_path + "/" + filename)
    #cleanup, remove file
    dbutils.fs.rm(dbfs_pdf_path + filename)

    return(blob_pdf_path + "/" + filename)

In [0]:
account_key = ""

In [0]:
search_action = "upload"

In [0]:
print(type(df_silver))

In [0]:
def customFunction(row):
   return (row.path_to_pdf_in_storage, row.number_pages_pdf, row.pdflink_on_website)

silver_pdf_paths = df_silver.rdd.map(customFunction)

In [0]:
print(type(silver_pdf_paths))

In [0]:
silver_pdf_paths.collect()

In [0]:
#https://stackoverflow.com/questions/62733213/azure-blobstore-how-can-i-read-a-file-without-having-to-download-the-whole-thin

In [0]:
j = 1

In [0]:
for pdf_info in silver_pdf_paths.collect():
    j = j + 1
    #print(pdf_info)
    filename = pdf_info[0].rsplit('/', 1)[-1]
    print(filename)
    filepath_http =  "https://storage2023jti.blob.core.windows.net/testcontainer/pdfs_permanent_storage/" + filename + "?" + account_key
    #print(filepath_http)
    #dbutils.fs.cp(pdf_info[0], dbfs_pdf_path + filename)
    filepath_dbfs = "/dbfs/FileStore/pdfs_temp/"
    for i in range(1,pdf_info[1]):
        doc_number = str(j)
        print("doc number " + str(j))
        page_number = str(i)
        print("page number " + str(i))
        doc_id = str(j) + str(i)
        print("doc id " + doc_id)
        try:
            markdown = convert_pdf_table_to_md(filepath_dbfs + filename , i)
            df_html = convert_pdf_table_to_html(filepath_dbfs + filename, i)
            text = convert_pdf_page_to_text(filepath_dbfs + filename, i)
            single_page_pdf_name = save_pdf_page_to_storage(filepath_dbfs + filename, i)
            copy_pdf_from_filestore_to_blob(single_page_pdf_name + '.pdf')
            if (markdown):
                dataType = "table"
            else:
                dataType = "text"
            #print(markdown)
            data = {'@search.action':search_action, 'id': doc_id, 'content':markdown, 'page_number':page_number, 'document_link': pdf_info[2], 'document_link_storage': pdf_info[0],'dataType': dataType, 'title' : filename , 'content_html':df_html , 'content_text' : text, 'document_filename' : single_page_pdf_name + '.pdf'}
            #print(data)
            upload = json.dumps(data)
            indexDocument = json.loads(upload)
            try:
                result = search_client.upload_documents(documents=indexDocument)
                print("Upload of new document succeeded: {}".format(result[0].succeeded))
            except Exception as ex:
                print (ex.message)
        except Exception as ex:
                print (ex.message)
        

for i in range(1,84):
    print(i)
    page_number = str(i)
    markdown = convert_pdf_table_to_md(pdf_path, i)
    df_html = convert_pdf_table_to_html(pdf_path, i)
    #print(markdown)
    data = {'@search.action':search_action, 'id': str(i), 'content':markdown, 'page_number':page_number, 'document_link': "https://www.imf.org/-/media/Files/Publications/CR/2022/English/1BHSEA2022001.ashx",
           'dataType': dataType, 'title' : title , 'content_html':df_html }
    #print(data)
    table_upload = json.dumps(data)
    indexDocument = json.loads(table_upload)
    try:
        result = search_client.upload_documents(documents=indexDocument)
        print("Upload of new document succeeded: {}".format(result[0].succeeded))
    except Exception as ex:
        print (ex.message)

In [0]:
#test
results_GDP_bahamas = search_client.search(search_text="GDP Bahamas 2023")

for result in results_GDP_bahamas:
    print("{}: {})".format(result["id"], result["@search.score"]))


In [0]:
import openai
import time

In [0]:
openai.api_key = ""

In [0]:
def create_prompt (df_from_pdf_page):
    prompt_table_from_pdf = f""" Look into the table provided in triple backticks and find out the following information\
    1. Real GDP in 2023(%Change)
    2. GDP in 2023 (millions of Bahamian Dollars)
    3. Government Debt in 2023 (% of GDP)
    .Do not hallucinate. ```{df_from_pdf_page}```"""
    return (prompt_table_from_pdf)

In [0]:
def get_answer_from_openai (prompt_pdf_table):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages=[
            #{"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
            {"role": "user", "content": prompt_pdf_table}
            ]
        )
    return(response['choices'][0]['message']['content'])

In [0]:
results_unemployment_rate = search_client.search(search_text="Unemployment Rate")
i = 0
for result in results_unemployment_rate:
    if i > 3:
        break
    prompt_temp = create_prompt_unemployment(result["content"])
    print(prompt_temp)
    answer = get_answer_from_openai(prompt_temp)
    print(answer)
    time.sleep(60)
    i=i+1
    print("{}: {})".format(result["id"], result["@search.score"]))
